In [15]:
import pandas as pd
import datetime as dt
import re

df = pd.read_csv("que-faire-a-paris-.csv", sep=	";")

In [16]:
# Categorise columns according to whether or not to use them: "Yes", "No", "Maybe"
columns_to_keep = {
    "ID" : "Yes",
    "URL" : "Yes",
    "Titre" : "Yes",
    "Chapeau" : "Yes",
    "Description" : "Later",
    "Date de début" : "Yes",
    "Date de fin" : "Yes",
    "Occurrences" : "Maybe",
    "Description de la date" : "Maybe",
    "URL de l'image" : "No",
    "Texte alternatif de l'image" : "No",
    "Crédit de l'image" : "No",
    "Mots clés" : "Yes",
    "Nom du lieu" : "Yes",
    "Adresse du lieu" : "Yes",
    "Code postal" : "Yes",
    "Ville" : "Yes",
    "Coordonnées géographiques" : "Yes",
    "Accès PMR" : "Yes",
    "Accès mal voyant" : "Yes",
    "Accès mal entendant" : "Yes",
    "Transport" : "Later",
    "Url de contact" : "No",
    "Téléphone de contact" : "No",
    "Email de contact" : "No",
    "URL Facebook associée" : "No",
    "URL Twitter associée" : "No",
    "Type de prix" : "Yes",
    "Détail du prix" : "Maybe",
    "Type d'accès" : "Later",
    "URL de réservation" : "Yes",
    "URL de réservation - Texte" : "No",
    "Date de mise à jour" : "Yes",
    "Image de couverture" : "No",
    "Programmes" : "Maybe",
    "En ligne - address_url" : "No",
    "En ligne - address_url_text" : "No",
    "En ligne - address_text" : "No",
    "title_event" : "No",
    "audience" : "Yes",
    "childrens" : "No",
    "group" : "No",
    "locale" : "No",
    "rank" : "No",
    "weight" : "No"
}

In [17]:
# Filter the dataset to only have the columns that are marked "Yes" in the dictionary "columns_to_keep"
df_filtered = df[[col for col in df.columns if columns_to_keep[col] == "Yes"]]

In [18]:
# Drop duplicates
df_filtered = df_filtered.drop_duplicates()

In [19]:
# Impute missing values in "Titre", "Chapeau", "Mots clés", "Nom de lieu", "Adresse du lieu", "Détail du prix", "URL de réservation" with "N/A"
df_filtered["Titre"] = df_filtered["Titre"].fillna("N/A")
df_filtered["Chapeau"] = df_filtered["Chapeau"].fillna("N/A")
df_filtered["Mots clés"] = df_filtered["Mots clés"].fillna("N/A")
df_filtered["Nom du lieu"] = df_filtered["Nom du lieu"].fillna("N/A")
df_filtered["Adresse du lieu"] = df_filtered["Adresse du lieu"].fillna("N/A")
df_filtered["URL de réservation"] = df_filtered["URL de réservation"].fillna("N/A")

In [20]:
# Impute missing values in "Code postal" with "75000"
df_filtered["Code postal"] = df_filtered["Code postal"].fillna("75000")

In [21]:
# Impute missing values in "Ville" with "Paris"
df_filtered["Ville"] = df_filtered["Ville"].fillna("Paris")

In [22]:
# changer les données de la colonne 'Accès PMR' si = 1 alors 'Accès PMR' sinon NULL
df_filtered['Accès PMR'] = df_filtered['Accès PMR'].apply(lambda x: 'Accès PMR' if x == 1 else "Pas d'accès PMR")
df_filtered['Accès mal voyant'] = df_filtered['Accès mal voyant'].apply(lambda x: 'Accès mal voyant' if x == 1 else "Pas d'accès mal voyant")
df_filtered['Accès mal entendant'] = df_filtered['Accès mal entendant'].apply(lambda x: 'Accès mal entendant' if x == 1 else "Pas d'accès mal entendant")

In [23]:
# Regrouper les colonnes  'Accés PMR', 'Accés mal voyant' et Accés mal entendant' en une seule colonne 'Accessibilité' avec le détail des trois colonnes
df_filtered['Accessibilité'] = df_filtered['Accès PMR'] + ' – ' + df_filtered['Accès mal voyant'] + ' – ' + df_filtered['Accès mal entendant']

In [24]:
# Impute missing values in "Date de début"  with datetime.today() and missing values in "Date de fin" with the last date of the year
today = dt.datetime.today()
last_day_of_year = dt.datetime(today.year, 12, 31)
df_filtered["Date de début"] = df_filtered["Date de début"].fillna(today)
df_filtered["Date de fin"] = df_filtered["Date de fin"].fillna(last_day_of_year)

# Impute missing values in "Date de mise à jour" with the most recent date in the column
df_filtered["Date de mise à jour"] = df_filtered["Date de mise à jour"].fillna(df_filtered["Date de mise à jour"].max())


In [25]:
def clean_code_postal(code):
    # Strip spaces
    code = code.strip()
    # Replace all spaces with empty string
    code = re.sub(r'\s+', '', code)
    # Remove everything after the last number
    code = re.sub(r'\D+$', '', code)
    # If there are only 4 digits, add a leading zero
    if len(code) == 4:
        code = "0" + code
    # Use a regular expression to replace anything that is not a 5-digit number with the value "75000"
    if not re.match(r'^\d{5}$', code):
        code = "75000"
    return code

df_filtered["Code postal"] = df_filtered["Code postal"].apply(clean_code_postal)

In [26]:
# Pipeline for treating the "Ville" column
def clean_ville(ville):
    # Strip spaces at the end or beginning
    ville = ville.strip()
    # If the string starts with "Paris", replace it with "Paris"
    if ville.startswith("Paris"):
        ville = "Paris"
    # Convert all values to title case
    ville = ville.title()
    return ville

df_filtered["Ville"] = df_filtered["Ville"].apply(clean_ville)

In [27]:
# Convert "Type de prix" to title case
df_filtered["Type de prix"] = df_filtered["Type de prix"].str.title()